In [1]:
import jax

import jax.numpy as np
import jax.scipy as sp

from numpy.polynomial.hermite import hermgauss
import scipy as osp

from vb_lib import structure_model_lib, data_utils, cavi_lib, fast_structure_lib, plotting_utils
from vb_lib.preconditioner_lib import get_mfvb_cov_matmul

import paragami

from copy import deepcopy

import time

import matplotlib.pyplot as plt
%matplotlib inline  

from bnpmodeling_runjingdev import cluster_quantities_lib, modeling_lib


/accounts/grad/runjing_liu/.conda/envs/bnp_sensitivity_jax/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
import numpy as onp
onp.random.seed(53453)

# Load data

In [3]:
data_file = '../../../../fastStructure/test/testdata.npz'
data = np.load(data_file)
g_obs = np.array(data['g_obs'])

g_obs = np.array(data['g_obs'])
g_obs_raw = np.array(data['g_obs_raw'])

# just checking ... 
which_missing = (g_obs_raw == 3)
(g_obs.argmax(-1) == g_obs_raw)[~which_missing].all()
(g_obs[which_missing] == 0).all()

DeviceArray(True, dtype=bool)

In [4]:
n_obs = g_obs.shape[0]
n_loci = g_obs.shape[1]

# Load fastStructure fit

In [5]:
filename = '../../../../fastStructure/fits/test/testoutput_simple.3'

In [6]:
vb_params_dict, _ = fast_structure_lib.load_fs_to_vb_params(filename)

In [7]:
e_ind_admix, e_pop_freq = plotting_utils.get_vb_expectations(vb_params_dict)

### Check against fast-structure saved means

In [8]:
fs_e_ind_admix, _, fs_e_pop_freq, _ = \
    fast_structure_lib.load_fs_means_vars(filename)

In [9]:
np.abs(e_pop_freq - fs_e_pop_freq).max()

DeviceArray(5.01246185e-07, dtype=float64)

In [13]:
np.abs(e_ind_admix - fs_e_ind_admix).max()

DeviceArray(4.99500499e-07, dtype=float64)